In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run, ir_datasets
from tira.rest_api_client import Client
import pyterrier as pt
from pyterrier_pisa import PisaIndex

In [3]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# We are using Pisa Index to index the dataset
index = PisaIndex('./index', overwrite=True)

index.index(pt_dataset.get_corpus_iter())

# get all topics of training dataset
#topics = pt_dataset.get_topics()
#topics

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 0/126958 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pyterrier_pisa/__init__.py:144: UserWarning: text_field not specified; indexing all str fields: ['text']
  warn(f'text_field not specified; indexing all str fields: {text_field}')
/usr/local/lib/python3.10/dist-packages/pyterrier_pisa/indexers.py:38: UserWarning: Removing index
  warn(f'Removing {str(path)}')


[2024-06-15 13:12:12.407] [info] [Batch 1] Processed documents [100000, 126958)
[2024-06-15 13:12:26.592] [info] [Batch 0] Processed documents [0, 100000)
[2024-06-15 13:12:26.640] [info] Merging titles
[2024-06-15 13:12:26.662] [info] Creating document lexicon
[2024-06-15 13:12:26.678] [info] Merging URLs
[2024-06-15 13:12:26.680] [info] Collecting terms
[2024-06-15 13:12:26.718] [info] Writing terms
[2024-06-15 13:12:26.738] [info] Mapping terms
[2024-06-15 13:12:26.751] [info] Remapping IDs
[2024-06-15 13:12:27.145] [info] Concatenating batches
[2024-06-15 13:12:27.580] [info] Success.
[2024-06-15 13:12:27.605] [info] Inverting [0, 100000)
[2024-06-15 13:12:29.447] [info] Inverting [100000, 126958)
[2024-06-15 13:12:30.357] [info] Number of terms: 100128
[2024-06-15 13:12:30.357] [info] Number of documents: 126958
[2024-06-15 13:12:30.357] [info] Number of postings: 7914234


PisaIndex('./index')

# We now do Query expansion in order to improve retrieval effectiveness 

In [7]:
bm25 = index.bm25()

#(bm25 >> pt.text.get_text(pt_dataset, 'text')).search('retrieval')
pt.Experiment([bm25], pt_dataset.get_topics(), pt_dataset.get_qrels(), eval_metrics=['ndcg_cut_5'], names=['BM25'])

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut_5
0,BM25,0.479671


In [11]:
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

bo1_expansion = bm25 >> pt.rewrite.Bo1QueryExpansion(index)
bm25_bo1 = bo1_expansion >> bm25
pt.Experiment([bm25_bo1], pt_dataset.get_topics(), pt_dataset.get_qrels(), eval_metrics=['ndcg_cut_5'], names=['BM25 >> Bo1 >> BM25'])

ValueError: index_location is current a <class 'pyterrier_pisa.PisaIndex'>,
        while it needs to be an Index, an IndexRef, a string that can be
        resolved to an index location (e.g. path/to/index/data.properties),
        or an pyterrier.index.TerrierIndexer object

In [13]:
pt_dataset1 = ir_datasets.load('ir-lab-sose-2024/ir-acl-anthology-20240504-training')
print('The dataset has', len(list(pt_dataset1.docs_iter())), 'documents.')
for query in list(pt_dataset1.queries_iter())[:3]:
    print('\nQuery: ', query.query_id)

    print('\tText:\t\t' + query.default_text())
    print('\tDescrition:\t' + query.description)
    print('\tNarrative:\t' + query.narrative)

The dataset has 126958 documents.

Query:  1
	Text:		retrieval system improving effectiveness
	Descrition:	What papers focus on improving the effectiveness of a retrieval system?
	Narrative:	Relevant papers include research on what makes a retrieval system effective and what improves the effectiveness of a retrieval system. Papers that focus on improving something else or improving the effectiveness of a system that is not a retrieval system are not relevant.

Query:  2
	Text:		machine learning language identification
	Descrition:	What papers are about machine learning for language identification?
	Narrative:	Relevant papers include research on methods of machine learning for language identification or how to improve those methods. Papers that focus on other methods for language identification or the usaged of machine learning not for language identification are not relevant.

Query:  3
	Text:		social media detect self-harm
	Descrition:	Which papers focus on how to recognize signs of s